<a href="https://colab.research.google.com/github/Komsomolochka/sentiment_analysis/blob/main/sentiment_analysis_week6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [303]:
import requests
import bs4
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.svm import LinearSVC
from nltk.corpus import stopwords
import numpy as np
import pandas as pd


In [132]:
# создаем парсер
X = []
y = []
for page in range(1,50): #на каждой странице каиалога выбираем названия моделей
  url = 'https://hi-tech.mail.ru/mobile_android-catalog/?param5333=68555&page={}'.format(page)
  req = requests.get(url)
  soup = bs4.BeautifulSoup(req.text, 'lxml')
  names = soup.findAll('a', attrs={'class':'p-catalog-card__link link-holder'})
  names = list(set(names))
  for name in names: #собираем отзывы на каждую модель
    print(name['href'])
    url = 'https://hi-tech.mail.ru{}otzyvy/#feedback'.format(name['href'])
    req = requests.get(url)
    soup = bs4.BeautifulSoup(req.text, 'lxml')
    reviews = soup.findAll('div', attrs={'class':'p-model-review__text'})
    ratings = soup.findAll('div', attrs={'class':'p-model-review__stars'})

    for review in reviews:
      try:
        review.find(text="Плюсы").replaceWith("")
      except:
        pass
      try:
        review.find(text="Впечатления").replaceWith("")
      except:
        pass
      try:
        review.find(text="Минусы").replaceWith("")
      except:
        pass
      X.append(review.text)

    for rating in ratings:
      y.append(int(rating.text))


/xiaomi_mi_10t_8_128gb_14717229-catalog/
/xiaomi_poco_x3_nfc_6_128gb_14717821-catalog/
/samsung_galaxy_s21_8_128gb_14725009-catalog/
/nokia_5_4_4_64gb_14723168-catalog/
/huawei_mate_40_pro_8_256gb_14720013-catalog/
/samsung_galaxy_z_fold_2_12_256gb_14716835-catalog/
/xiaomi_redmi_9a_2_32gb_14710933-catalog/
/honor_10x_lite_4_128gb_14720129-catalog/
/oppo_a53_4_64gb_14717954-catalog/
/xiaomi_poco_m3_4_64gb_14723829-catalog/
/xiaomi_poco_m3_4_128gb_14723831-catalog/
/nokia_5_4_6_64gb_14723772-catalog/
/nokia_c1_plus_1_16gb_14723174-catalog/
/honor_20e_4_64gb_14714174-catalog/
/samsung_galaxy_s21_8_256gb_14725010-catalog/
/honor_30i_4_128gb_14716170-catalog/
/xiaomi_mi_11_8_256gb_14726501-catalog/
/oppo_a53_4_128gb_14717956-catalog/
/nokia_5_4_4_128gb_14723770-catalog/
/samsung_galaxy_s21_8_128gb_14725006-catalog/
/samsung_galaxy_s21_8_256gb_14725007-catalog/
/asus_rog_phone_3_12_256gb_14715734-catalog/
/oppo_reno4_pro_5g_12_256gb_14717302-catalog/
/oppo_reno4_lite_8_128gb_14717301-catalo

In [135]:
# сохраняем результат
train_data = pd.DataFrame()
train_data['review'] = X
train_data['rating'] = y
train_data.to_csv("train_data.csv")

In [315]:
# удаляем отзывы с рейтингом 3, 1 и 2 меняем на 0, 4 и 5 на 1
train_data = train_data.loc[train_data['rating']!=3]
train_data['rating'] = train_data['rating'].apply(lambda x: 0 if x in [1,2] else 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [316]:
# импортируем тестовые данные
import codecs
with codecs.open('test.csv', 'rb', encoding='utf8') as f: # придется применить кодирование в utf8
    data = f.read()

test = [review.text for review in bs4.BeautifulSoup(data, 'lxml').find_all('review')]

In [317]:
texts = list(train_data['review'])
labels = list(train_data['rating'])

In [318]:
# чистим текст
texts = [i.replace('\n', ' ').replace('\r', '').replace('\t', ' ').replace('<br />', ' ').replace('<br>', ' ')
               if len(i.split()) > 1
               else i.replace('\n', ' ').replace('\r', '').replace('\t', ' ').replace('<br />', ' ').replace('<br>', ' ')
               for i in texts]

In [319]:
# удаляем дубликаты
train_data.drop_duplicates(inplace = True)

In [322]:
# удаляем отзывы, содержащие только 1 слово
d = {item: idx  for idx, item in enumerate(list(train_data.review)) if len(item.split())==1 }
train_data.drop(train_data.index[list(d.values())], inplace=True)

In [323]:
# проверим баланс классов
train_data['rating'].value_counts()

1    2858
0     208
Name: rating, dtype: int64

In [324]:
# балансируем классы
train_data = pd.concat([train_data[train_data['rating'] == 1], 
                        train_data[train_data['rating'] == 0],
                        train_data[train_data['rating'] == 0],
                        train_data[train_data['rating'] == 0],
                        train_data[train_data['rating'] == 0],
                        train_data[train_data['rating'] == 0],
                        train_data[train_data['rating'] == 0],
                        train_data[train_data['rating'] == 0],
                        train_data[train_data['rating'] == 0],
                        train_data[train_data['rating'] == 0],
                        train_data[train_data['rating'] == 0],
                        train_data[train_data['rating'] == 0],
                        train_data[train_data['rating'] == 0],
                        train_data[train_data['rating'] == 0],
]
                      )

In [325]:
train_data['rating'].value_counts()

1    2858
0    2704
Name: rating, dtype: int64

In [326]:
# строим модели
vectorizers = [TfidfVectorizer, CountVectorizer]
classifiers = [LogisticRegression, LinearSVC]

for c in classifiers:
    for v in vectorizers:
        model = Pipeline([("vectorizer", v()), ("classifier", c())])
        print(str(c().__class__.__name__) + ' ' + str(v().__class__.__name__))
        print(cross_val_score(model, texts, labels, scoring='accuracy', cv = 5).mean())
        print

LogisticRegression TfidfVectorizer
0.9308855979759935
LogisticRegression CountVectorizer
0.9339729688599745
LinearSVC TfidfVectorizer
0.9358233940154843
LinearSVC CountVectorizer
0.9253314691167798


In [327]:
# добавляем n-граммы
for c in classifiers:
    for v in vectorizers:
        model = Pipeline([("vectorizer", v(ngram_range=(1, 2))), ("classifier", c())])
        print(str(c().__class__.__name__) + ' ' + str(v().__class__.__name__))
        print(cross_val_score(model, texts, labels, scoring='accuracy', cv = 5).mean())
        print

LogisticRegression TfidfVectorizer
0.9308855979759935
LogisticRegression CountVectorizer
0.9358229184500372
LinearSVC TfidfVectorizer
0.9385997450969203
LinearSVC CountVectorizer
0.931503357492058


In [328]:
  import nltk
  nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [329]:
# добавляем стоп-слова
for c in classifiers:
    for v in vectorizers:
        model = Pipeline([("vectorizer", v(ngram_range=(1, 2), stop_words=stopwords.words('russian'))), ("classifier", c())])
        print(str(c().__class__.__name__) + ' ' + str(v().__class__.__name__))
        print(cross_val_score(model, texts, labels, scoring='accuracy', cv = 5).mean())
        print

LogisticRegression TfidfVectorizer
0.931194239951302
LogisticRegression CountVectorizer
0.9311942399513022
LinearSVC TfidfVectorizer
0.933044665106812
LinearSVC CountVectorizer
0.930577431566132


In [330]:
# подбираем параметры
clf = Pipeline([('vectorizer', TfidfVectorizer()), ('classifier', LinearSVC())])

In [331]:
params = {
    'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'vectorizer__stop_words': [stopwords.words('russian'), None],
    'classifier__C': [0.9, 1.0, 1.1, 1.2]
}

In [332]:
grid_cv = RandomizedSearchCV(clf, params, n_iter = 1000, scoring = 'accuracy', n_jobs=-1)

In [333]:
%%time
grid_cv.fit(texts, labels)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 24 is smaller than n_iter=1000. Running 24 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


CPU times: user 5.66 s, sys: 144 ms, total: 5.81 s
Wall time: 29.7 s


RandomizedSearchCV(estimator=Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                                             ('classifier', LinearSVC())]),
                   n_iter=1000, n_jobs=-1,
                   param_distributions={'classifier__C': [0.9, 1.0, 1.1, 1.2],
                                        'vectorizer__ngram_range': [(1, 1),
                                                                    (1, 2),
                                                                    (1, 3)],
                                        'vectorizer__stop_words': [['и', 'в',
                                                                    'во', 'не',
                                                                    'что', 'он',
                                                                    'на', 'я',
                                                                    'с', 'со',
                                                                    'как', 'а',
                

In [334]:
print(grid_cv.best_params_)

{'vectorizer__stop_words': None, 'vectorizer__ngram_range': (1, 2), 'classifier__C': 1.2}


In [335]:
print(grid_cv.best_score_)

0.9389088626376763


In [336]:
pred = grid_cv.best_estimator_.fit(texts,labels).predict(test)

In [337]:
sub = pd.Series(pred).apply(lambda x: 'neg' if x == 0 else 'pos')
df = sub.to_frame()
df.columns = ["y"]
df.index.name = "Id"
df.to_csv("submission.csv")